In [44]:
import pandas as pd
import numpy as np

In [45]:
# adding programming, author, Year to train --  done in excel because excel uses multiple processors to do this
# adding programming and author to test as well. It can be easily done in the code by looking up but currently my computing power is low
# I have sample code below

In [46]:
challenge = pd.read_csv("challenge_data.csv") # read challenge data set
users_data_train = pd.read_csv("train.csv") # train data
test = pd.read_csv("test.csv")

C:\Users\320006523\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
# popular categories based on programming language and year

# get selected columns
temp = challenge[["challenge_ID","programming_language","publish_date","author_ID"]]

# Get year from date
temp["Year"] = pd.to_datetime(temp.loc[:,"publish_date"]).dt.year

# drop publish date
temp = temp.drop(["publish_date"], axis = 1)

# Target columns
temp["total_submissions"] = challenge["total_submissions"]

# sort the target
temp = temp.sort_values(by=["total_submissions"], ascending= False)
temp.index = np.arange(0,len(temp))

C:\Users\320006523\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [48]:
temp.head()

,challenge_ID,programming_language,author_ID,Year,total_submissions
0,CI24917,1,AI564588,2007,43409.0
1,CI23855,1,AI563823,2006,20993.0
2,CI24261,1,AI564133,2005,20674.0
3,CI23933,1,AI563783,2006,15086.0
4,CI23714,1,AI563766,2006,14723.0


In [49]:
users_data_train.head()

,user_sequence,user_id,challenge_sequence,challenge,programming,author_org_ID
0,4576_1,4576,1,CI23714,1,AOI100129
1,4576_2,4576,2,CI23855,1,0
2,4576_3,4576,3,CI24917,1,AOI100581
3,4576_4,4576,4,CI23663,1,AOI100098
4,4576_5,4576,5,CI23933,1,AOI100149


In [50]:
test.head()

,user_sequence,user_id,challenge_sequence,challenge,programming,Author,Year
0,4577_1,4577,1,CI23855,1,AI563823,2006
1,4577_2,4577,2,CI23933,1,AI563783,2006
2,4577_3,4577,3,CI24917,1,AI564588,2007
3,4577_4,4577,4,CI24915,1,AI563790,2007
4,4577_5,4577,5,CI23714,1,AI563766,2006


In [51]:
# Sample code to add programming to train. SImilarly we can add other features to train and test as well
# for x in range(len(users_data_train)):
#     print(x)
#     users_data_train.loc[x,"Programming"] = temp.loc[temp["challenge_ID"] == users_data_train.loc[x,"challenge"],"programming_language"].values[0] 

In [52]:
# get the weight for programming. Recent value is more important
weight = []
for i in range(10):
    weight.append(0.1*i)

In [53]:
# create result array using popularity(total) index
result = pd.DataFrame(columns=["user_sequence","challenge"]) # Taret Sample
val = 0 # for popular array
count = 0 # for result array
for x in range(int(len(test)/10)): # For N users
    user_challenge = test.loc[x*10:((x+1)*10) - 1, "challenge"]
    program_pref = np.argmax(np.bincount(test.loc[x*10:((x+1)*10) - 1,"programming"],weights = weight)) # get the value for programming which is max
#     program_count = np.max(np.bincount(test.loc[x*10:((x+1)*10) - 1,"programming"])) # get the count value
    length_author = len(np.unique(test.loc[x*10:((x+1)*10) - 1,"Author"])) # get the number of author
    min_year_pref = np.unique(test.loc[x*10:((x+1)*10) - 1, "Year"])[0] # min year preference
    year_length = len(np.unique(test.loc[x*10:((x+1)*10) - 1, "Year"])) # specific year preference
    
    
    val = 0
    count = 0
    while count!=3: # For forecasting 3 values
        result.loc[x*3 + count, "user_sequence"] = str(test.loc[x*10, "user_id"]) + "_" + str(count+11)
        
        # filter popular array by using program pref and author
        popular = temp[temp["programming_language"]==program_pref]
        
        if year_length == 1:
            popular = popular[popular["Year"] == min_year_pref]        
        
        if length_author == 1:
            popular = popular[popular["author_ID"] == test.loc[x*10,"Author"]]
        
        popular.index = np.arange(0,len(popular))
        
        if len(popular)<15:
            popular = pd.concat([popular,temp.loc[temp["programming_language"]==program_pref,:]])
            popular.index = np.arange(0,len(popular))
        
        # Check whether user has already taken the challenge or not
        if len(user_challenge[user_challenge == popular.loc[val,"challenge_ID"]]) == 0:
            result.loc[x*3 + count, "challenge"] = popular.loc[val,"challenge_ID"]
            val += 1
            count = count + 1
        else:
            val += 1

In [54]:
result.to_csv("Final_Results2.csv", index=False)

In [ ]:
# could have done similar user analysis as well to increase the accuracy. But due to my slow machine did not get much time to do
# Not an excuse. But this is my first attempt. I really want to be at Mckinsey and looking for Full time as a Data Scientist in
# United States. Please contact me at 7747012186 or email me at dasanghavi@wpi.edu